# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
import json
import requests
from key import weather_api_key
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import scipy.stats as st
from scipy.stats import linregress


# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy


# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes for whole earth
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [16]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
#print(cities)
len(cities)

616

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
base_url = "https://api.openweathermap.org/data/2.5/weather?"


record = 0
sets = 1
query_url = base_url + "appid=" + weather_api_key + "&q=" + city
response = requests.get(query_url).json()
print(json.dumps(response, indent=4, sort_keys=True))

In [44]:
base_url = "https://api.openweathermap.org/data/2.5/weather?"


record = 0
sets = 1

weather_df = pd.DataFrame({"City": cities})
weather_df["Country"]=""
weather_df["Date"]= ""
weather_df["Latitude"] = ""
weather_df["Longitude"] = ""
weather_df["Cloudiness (%)"] = ""
weather_df["Wind Speed (mph)"] = ""
weather_df["Max Temp (F)"] = ""
weather_df["Humidity (%)"] = ""
for index, row in weather_df.iterrows():
    city = row['City']
    query_url = base_url + "appid=" + weather_api_key + "&q=" + city
    response = requests.get(query_url).json()
    try:
        print(f"Processing Record {record}  of Set {sets} | {city}.")

        weather_df.loc[index, "Country"] = response["sys"]["country"]     
        weather_df.loc[index, "Date"] = datetime.datetime.fromtimestamp(response["dt"])
        weather_df.loc[index, "Latitude"] = response["coord"]["lat"]
        weather_df.loc[index, "Longitude"] = response["coord"]["lon"]
        weather_df.loc[index, "Cloudiness (%)"] = response["clouds"]["all"]
        weather_df.loc[index, "Humidity (%)"] = response["main"]["humidity"]
        weather_df.loc[index, "Max Temp (F)"] = 1.8 * (response["main"]["temp_max"] - 273) + 32
        weather_df.loc[index, "Wind Speed (mph)"] = response["wind"]["speed"]
    except:
        print(f"City not found...skipping {city}.") 
        #print (query_url) 
        
    
    record = record + 1
    if record == 50:
        record = 0
        sets = sets + 1 

print("-----------------------------")        
print("Data Retrieval Complete")
print("-----------------------------")   
    


Processing Record 0  of Set 1 | kholmogory.
Processing Record 1  of Set 1 | lebu.
Processing Record 2  of Set 1 | ponta do sol.
Processing Record 3  of Set 1 | hilo.
Processing Record 4  of Set 1 | cherskiy.
Processing Record 5  of Set 1 | ust-kamchatsk.
City not found...skipping ust-kamchatsk.
Processing Record 6  of Set 1 | salo.
Processing Record 7  of Set 1 | lucapa.
Processing Record 8  of Set 1 | vaitupu.
City not found...skipping vaitupu.
Processing Record 9  of Set 1 | gao.
Processing Record 10  of Set 1 | punta arenas.
Processing Record 11  of Set 1 | rikitea.
Processing Record 12  of Set 1 | kavieng.
Processing Record 13  of Set 1 | barentsburg.
City not found...skipping barentsburg.
Processing Record 14  of Set 1 | tasiilaq.
Processing Record 15  of Set 1 | tiksi.
Processing Record 16  of Set 1 | qaanaaq.
Processing Record 17  of Set 1 | anchorage.
Processing Record 18  of Set 1 | umzimvubu.
City not found...skipping umzimvubu.
Processing Record 19  of Set 1 | bambous virieu

In [45]:
weather_df


,City,Country,Date,Latitude,Longitude,Cloudiness (%),Wind Speed (mph),Max Temp (F),Humidity (%)
0,kholmogory,RU,2022-04-23 16:38:46,64.2229,41.656,100,3.04,35.798,91
1,lebu,CL,2022-04-23 16:38:47,-37.6167,-73.65,100,7.21,57.254,97
2,ponta do sol,PT,2022-04-23 16:38:48,32.6667,-17.1,71,3.96,62.402,56
3,hilo,US,2022-04-23 16:38:50,19.7297,-155.09,20,2.57,72.518,93
4,cherskiy,RU,2022-04-23 16:38:51,68.75,161.3,93,0.77,18.266,95
...,...,...,...,...,...,...,...,...,...
611,matadi,CD,2022-04-23 16:52:07,-5.8167,13.45,78,1.42,85.892,70
612,sooke,CA,2022-04-23 16:52:09,48.3746,-123.728,28,9.26,48.452,82
613,blackwater,AU,2022-04-23 16:52:10,-23.5833,148.883,93,6.01,74.282,52
614,zhanatas,,,,,,,,


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression